In [3]:
import pandas as pd

# Load the uploaded file
file_path = '女子网球q2指标.xlsx'
data = pd.read_excel(file_path)

data

,比赛ID,已进行时间,盘数,局数,球员1赢得盘数,球员2赢得盘数,球员1赢得局数,球员2赢得局数,此分中的回合数（包括发球,1波动,2波动,球员1连胜盘数,球员2连胜盘数,球员1连胜局数,球员2连胜局数,1获胜盘分差,2获胜盘分差,1获胜局分差,2获胜局分差
0,2023-wimbledon-2503,00:02:15,1,1,0,0,0,0,1,1,4,0,0,0,0,0,0,0,0
1,2023-wimbledon-2503,00:04:05,1,2,0,0,0,1,7,4,0,0,0,0,1,0,0,-1,1
2,2023-wimbledon-2503,00:06:34,1,3,0,0,1,1,5,1,4,0,0,1,0,0,0,0,0
3,2023-wimbledon-2503,00:10:14,1,4,0,0,1,2,2,0,4,0,0,0,1,0,0,-1,1
4,2023-wimbledon-2503,00:13:04,1,5,0,0,1,3,2,4,0,0,0,0,2,0,0,-2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2023-wimbledon-2701,01:07:25,2,6,1,0,2,3,9,4,2,0,0,1,0,1,-1,-1,1
114,2023-wimbledon-2701,01:10:49,2,7,1,0,3,3,5,2,4,0,0,2,0,1,-1,0,0
115,2023-wimbledon-2701,01:13:56,2,8,1,0,3,4,1,4,1,0,0,0,1,1,-1,-1,1
116,2023-wimbledon-2701,01:16:33,2,9,1,0,4,4,4,4,2,0,0,1,0,1,-1,0,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np



# 提取相关的评价指标列
criteria = data.iloc[:, 9:19]

# 权重设置
weights = np.array([0.34, 0.10, 0.23, 0.10, 0.23])  # 对应波动、连胜盘分数、连胜局分数、分差盘得分、分差局得分

# 分别处理选手1和选手2的数据
criteria_p1 = criteria.iloc[:, [0, 2, 4, 6, 8]]  # 选手1的指标
criteria_p2 = criteria.iloc[:, [1, 3, 5, 7, 9]]  # 选手2的指标

# 标准化处理
scaler = MinMaxScaler()
criteria_p1_scaled = scaler.fit_transform(criteria_p1)
criteria_p2_scaled = scaler.fit_transform(criteria_p2)

# 构建加权标准化矩阵
weighted_criteria_p1 = criteria_p1_scaled * weights
weighted_criteria_p2 = criteria_p2_scaled * weights

# 计算正理想解和负理想解
positive_ideal_p1 = weighted_criteria_p1.max(axis=0)
negative_ideal_p1 = weighted_criteria_p1.min(axis=0)
positive_ideal_p2 = weighted_criteria_p2.max(axis=0)
negative_ideal_p2 = weighted_criteria_p2.min(axis=0)

# 计算与正理想解和负理想解的距离
distance_to_positive_p1 = np.sqrt(((weighted_criteria_p1 - positive_ideal_p1) ** 2).sum(axis=1))
distance_to_negative_p1 = np.sqrt(((weighted_criteria_p1 - negative_ideal_p1) ** 2).sum(axis=1))
distance_to_positive_p2 = np.sqrt(((weighted_criteria_p2 - positive_ideal_p2) ** 2).sum(axis=1))
distance_to_negative_p2 = np.sqrt(((weighted_criteria_p2 - negative_ideal_p2) ** 2).sum(axis=1))

# 计算相对接近度
relative_closeness_p1 = distance_to_negative_p1 / (distance_to_positive_p1 + distance_to_negative_p1)
relative_closeness_p2 = distance_to_negative_p2 / (distance_to_positive_p2 + distance_to_negative_p2)

# 结果整合
scores_p1_p2 = pd.DataFrame({
    "选手1评分": relative_closeness_p1,
    "选手2评分": relative_closeness_p2
})

# 将计算出的评分加到原数据的后两列
data_with_scores = data.copy()
data_with_scores['选手1势头评分'] = scores_p1_p2['选手1评分']
data_with_scores['选手2势头评分'] = scores_p1_p2['选手2评分']


output_file_path = '女子Q2指标_带势头评分.xlsx'
data_with_scores.to_excel(output_file_path, index=False)
